In [53]:
import pandas as pd

df=pd.read_csv(r"D:\data\WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.columns
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0}) # zmieniamy wartości na Boolean
df.dropna # odrzucamy wiersze z wartościami NaN

<bound method DataFrame.dropna of       Age  Attrition     BusinessTravel  DailyRate              Department  \
0      41          1      Travel_Rarely       1102                   Sales   
1      49          0  Travel_Frequently        279  Research & Development   
2      37          1      Travel_Rarely       1373  Research & Development   
3      33          0  Travel_Frequently       1392  Research & Development   
4      27          0      Travel_Rarely        591  Research & Development   
...   ...        ...                ...        ...                     ...   
1465   36          0  Travel_Frequently        884  Research & Development   
1466   39          0      Travel_Rarely        613  Research & Development   
1467   27          0      Travel_Rarely        155  Research & Development   
1468   49          0  Travel_Frequently       1023                   Sales   
1469   34          0      Travel_Rarely        628  Research & Development   

      DistanceFromHome  Educa

In [54]:
# zmienna określana Attrition
Y = df['Attrition'].values
X = df.drop('Attrition', axis=1).values

print(X.shape)

# sprwadzamy zbilansowanie zbioru
n_pos = (Y == 1).sum()
n_neg = (Y == 0).sum()

# not well balanced
print(f"Number of positive: {n_pos}, number of negative: {n_neg}")

(1470, 34)
Number of positive: 237, number of negative: 1233


In [79]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score,classification_report,roc_auc_score
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from imblearn.pipeline import Pipeline, make_pipeline

In [56]:
from sklearn.model_selection import train_test_split
#podział na zbiór trenignowy i testowy
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

In [57]:
from sklearn.preprocessing import OneHotEncoder

#zamiana cech jakoścowych na n cech binarnych

def one_hot_encode(X_train, X_test):
    enc = OneHotEncoder(handle_unknown='ignore')

    
    X_train_enc = enc.fit_transform(X_train)

    # Transform the testing data using the same encoder
    X_test_enc = enc.transform(X_test)

    return X_train_enc, X_test_enc

X_train_enc,X_test_enc=one_hot_encode(X_train, X_test)
print(X_train_enc.shape)

(1102, 4315)


In [58]:
from imblearn.over_sampling import RandomOverSampler

def data_oversampling()->tuple:

    name="Oversampling"

    oversample = RandomOverSampler(sampling_strategy=0.7,random_state=42)

    return(oversample,name)

In [59]:
from imblearn.under_sampling import RandomUnderSampler

def data_undersampling()->tuple:

    name="Undersampling"
    
    undersample = RandomUnderSampler(sampling_strategy=0.7,random_state=42)

    return (undersample,name)



In [60]:
from imblearn.over_sampling import SMOTE

def data_smote()-> tuple:

    name="SMOTE"
    smote=SMOTE(random_state=42)
   

    return (smote,name)

In [61]:


func_list=[data_oversampling,data_undersampling,data_smote]

In [65]:


def decison_tree(func_list: list,params: dict) -> None: 
    
    #ramka danych do przechowywania wyników
    df_score = pd.DataFrame(columns=['Random forest with','Recall','Precision','F1 Score', 'Accuracy','Roc Auc'])
    
    # iterujemy po wektorze z funkcjami
    for func in func_list:
        #tworzymy pipeline
        pipeline = make_pipeline(func()[0], 
                                  RandomForestClassifier(criterion='gini',n_estimators=100, random_state=42))
        
        #werfyikacja kryżowa
        new_params = {'randomforestclassifier__' + key: params[key] for key in params}
        grid_search = GridSearchCV(pipeline, param_grid=new_params, cv=3, scoring='f1',
                                return_train_score=True,n_jobs=-1)
        grid_search.fit(X_train_enc,Y_train)
        print(grid_search.best_params_)
        random_forest_best=grid_search.best_estimator_
        pos_prob=random_forest_best.predict_proba(X_test_enc)[:,1]


        pred=random_forest_best.predict(X_test_enc)
        
        #dokładnośc modelu
        cm = confusion_matrix(Y_test,pred)
        print(cm)
 
        roc_auc=roc_auc_score(Y_test,pos_prob)
        recall = recall_score(Y_test,pred)
        precision = precision_score(Y_test,pred)
        f1 = f1_score(Y_test,pred)
        accuracy = accuracy_score(Y_test,pred)
        
        #zapisujemy dane do ramki danych

        ndf = [func()[1],recall, precision, f1, accuracy,roc_auc]

        df_score=df_score.append(pd.Series(ndf, index=df_score.columns[:len(ndf)]), ignore_index=True)
        
    
    display(df_score)





In [64]:


def random_forest(func_list: list,params: dict) -> None: 
    
    #ramka danych do przechowywania wyników
    df_score = pd.DataFrame(columns=['Decison tree with','Recall','Precision','F1 Score', 'Accuracy','Roc Auc'])
    
    # iterujemy po wektorze z funkcjami
    for func in func_list:
        #tworzymy pipeline
        pipeline = make_pipeline(func()[0], 
                                  DecisionTreeClassifier(random_state=42))
        
        #werfyikacja kryżowa
        new_params = {'deciosntreeclassifier__' + key: params[key] for key in params}
        grid_search = GridSearchCV(pipeline, param_grid=new_params, cv=3, scoring='f1',
                                return_train_score=True,n_jobs=-1)
        grid_search.fit(X_train_enc,Y_train)
        print(grid_search.best_params_)
        decision_tree_best=grid_search.best_estimator_
        pos_prob=decision_tree_best.predict_proba(X_test_enc)[:,1]


        pred=decision_tree_best.predict(X_test_enc)
        
        #dokładnośc modelu
        cm = confusion_matrix(Y_test,pred)
        print(cm)
 
        roc_auc=roc_auc_score(Y_test,pos_prob)
        recall = recall_score(Y_test,pred)
        precision = precision_score(Y_test,pred)
        f1 = f1_score(Y_test,pred)
        accuracy = accuracy_score(Y_test,pred)
        
        #zapisujemy dane do ramki danych

        ndf = [func()[1],recall, precision, f1, accuracy,roc_auc]

        df_score=df_score.append(pd.Series(ndf, index=df_score.columns[:len(ndf)]), ignore_index=True)
        
    
    display(df_score)





In [ ]:
def random_forest(func_list: list,params: dict) -> None: 
    
    #ramka danych do przechowywania wyników
    df_score = pd.DataFrame(columns=['Random forest with','Recall','Precision','F1 Score', 'Accuracy','Roc Auc'])
    
    # iterujemy po wektorze z funkcjami
    for func in func_list:
        #tworzymy pipeline
        pipeline = make_pipeline(func()[0], 
                                  RandomForestClassifier(n_estimators=100, random_state=42))
        
        #werfyikacja kryżowa
        new_params = {'randomforestclassifier__' + key: params[key] for key in params}
        grid_search = GridSearchCV(pipeline, param_grid=new_params, cv=3, scoring='f1',
                                return_train_score=True,n_jobs=-1)
        grid_search.fit(X_train_enc,Y_train)
        print(grid_search.best_params_)
        random_forest_best=grid_search.best_estimator_
        pos_prob=random_forest_best.predict_proba(X_test_enc)[:,1]


        pred=random_forest_best.predict(X_test_enc)
        
        #dokładnośc modelu
        cm = confusion_matrix(Y_test,pred)
        print(cm)
 
        roc_auc=roc_auc_score(Y_test,pos_prob)
        recall = recall_score(Y_test,pred)
        precision = precision_score(Y_test,pred)
        f1 = f1_score(Y_test,pred)
        accuracy = accuracy_score(Y_test,pred)
        
        #zapisujemy dane do ramki danych

        ndf = [func()[1],recall, precision, f1, accuracy,roc_auc]

        df_score=df_score.append(pd.Series(ndf, index=df_score.columns[:len(ndf)]), ignore_index=True)
        
    
    display(df_score)


In [85]:
def svc(func_list: list,params: dict) -> None: 
    
    #ramka danych do przechowywania wyników
    df_score = pd.DataFrame(columns=['SVC with','Recall','Precision','F1 Score', 'Accuracy','Roc Auc'])
    
    # iterujemy po wektorze z funkcjami
    for func in func_list:
        #tworzymy pipeline
        pipeline = Pipeline(steps=[
            ('preprocess', preprocess),
            ('model', ())
        ])

        
        #werfyikacja kryżowa
        new_params = {'svcclassifier__' + key: params[key] for key in params}
        grid_search = GridSearchCV(pipeline, param_grid=new_params, cv=3, scoring='f1',
                                return_train_score=True,n_jobs=-1)
        grid_search.fit(X_train_enc,Y_train)
        print(grid_search.best_params_)
        svc_best=grid_search.best_estimator_
        pos_prob=svc_best.predict_proba(X_test_enc)[:,1]


        pred=svc_best.predict(X_test_enc)
        
        #dokładnośc modelu
        cm = confusion_matrix(Y_test,pred)
        print(cm)
 
        roc_auc=roc_auc_score(Y_test,pos_prob)
        recall = recall_score(Y_test,pred)
        precision = precision_score(Y_test,pred)
        f1 = f1_score(Y_test,pred)
        accuracy = accuracy_score(Y_test,pred)
        
        #zapisujemy dane do ramki danych

        ndf = [func()[1],recall, precision, f1, accuracy,roc_auc]

        df_score=df_score.append(pd.Series(ndf, index=df_score.columns[:len(ndf)]), ignore_index=True)
        
    
    display(df_score)


In [66]:
params_deciosn_tree = {'max_depth': [5,10,None],
                    'min_samples_split': [10, 30, 50],
                    'min_samples_leaf': [1, 2, 3]}

decison_tree(func_list,params_deciosn_tree)

{'randomforestclassifier__max_depth': None, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__min_samples_split': 50}
[[312   8]
 [ 40   8]]


C:\Users\karol\AppData\Local\Temp\ipykernel_15832\667122590.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_score=df_score.append(pd.Series(ndf, index=df_score.columns[:len(ndf)]), ignore_index=True)


{'randomforestclassifier__max_depth': None, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_samples_split': 10}
[[303  17]
 [ 36  12]]


C:\Users\karol\AppData\Local\Temp\ipykernel_15832\667122590.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_score=df_score.append(pd.Series(ndf, index=df_score.columns[:len(ndf)]), ignore_index=True)


{'randomforestclassifier__max_depth': 5, 'randomforestclassifier__min_samples_leaf': 3, 'randomforestclassifier__min_samples_split': 50}
[[299  21]
 [ 37  11]]


C:\Users\karol\AppData\Local\Temp\ipykernel_15832\667122590.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_score=df_score.append(pd.Series(ndf, index=df_score.columns[:len(ndf)]), ignore_index=True)


,Random forest with,Recall,Precision,F1 Score,Accuracy,Roc Auc
0,Oversampling,0.166667,0.500000,0.250000,0.869565,0.748503
1,Undersampling,0.250000,0.413793,0.311688,0.855978,0.759961
2,SMOTE,0.229167,0.343750,0.275000,0.842391,0.729036


In [ ]:
params_random_forest={'max_depth': [5, 10, None],
                    'min_samples_split': [10, 30, 50],
                    'min_samples_leaf': [1, 2, 3]}
random_forest(func_list,params_random_forest)

In [86]:
paramas_svc={"C": [0.25,0.5,1,2,3],
            "gamma":[1e-07,1e-08,1e-09],
            'kernel':['linear']}
svc(func_list,paramas_svc)

ValueError: Invalid parameter 'svcclassifier' for estimator Pipeline(steps=[('randomoversampler',
                 RandomOverSampler(random_state=42, sampling_strategy=0.7)),
                ('svc', SVC(probability=True, random_state=42))]). Valid parameters are: ['memory', 'steps', 'verbose'].